# Importing

In [1]:
#import cupy as cp
import pandas as pd
import numpy as np
from numba import jit

# Disable if running on GPU
from numba import jit, config
config.DISABLE_JIT = True
import cProfile
import sys
sys.path.append("../gaia_tools/")
import transformation_constants
import transformation_functions
import data_analysis
import covariance_generation as cov

In [2]:
# LOCAL
# print('Grabbing needed columns')
# icrs_data = pd.read_csv('/home/svenpoder/DATA/Gaia_2MASS Data_DR2/gaia_rv_data_bayes.csv', nrows = 10)
# print('Importing DR3')
# path = '/home/svenpoder/DATA/Gaia_DR3/GaiaDR3_RV_RGB_fidelity.csv'
# gaia_dr3 = pd.read_csv(path)
# icrs_data = gaia_dr3[icrs_data.columns]


# REMOTE
print('Grabbing needed columns')
icrs_data = pd.read_csv('/local/sven/gaia_tools_data/gaia_rv_data_bayes.csv', nrows = 10)
print('Importing DR3')
path = '/local/mariacst/2022_v0_project/data/GaiaDR3_RV_RGB_fidelity.csv'
gaia_dr3 = pd.read_csv(path, nrows=1e5)
icrs_data = gaia_dr3[icrs_data.columns]

Grabbing needed columns
Importing DR3


In [3]:
## TRANSFORMATION CONSTANTS
v_sun = transformation_constants.V_SUN

r_0 = 8277
z_0 = 25

v_sun[0][0] = 11.1
v_sun[1][0] = 251.5*(r_0/8277)
v_sun[2][0] = 8.59*(r_0/8277)

In [5]:
galcen_data = data_analysis.get_transformed_data(icrs_data,
                                       include_cylindrical = True,
                                       z_0 = z_0,
                                       r_0 = r_0,
                                       v_sun = v_sun,
                                       debug = True,
                                       is_bayes = True,
                                       is_source_included = True)

galcen_data = galcen_data[(galcen_data.r < 15000) & (galcen_data.r > 5000)]
galcen_data = galcen_data[(galcen_data.z < 200) & (galcen_data.z > -200)]
galcen_data.reset_index(inplace=True, drop=True)

icrs_data = icrs_data.merge(galcen_data, on='source_id')[icrs_data.columns]
print("Final size of sample {}".format(galcen_data.shape))
galcen_data.columns

Starting galactocentric transformation loop over all data points.. 
Time elapsed for data coordinate transformation: 0.2561889961361885 sec
Final size of sample (37921, 11)


Index(['x', 'y', 'z', 'v_x', 'v_y', 'v_z', 'r', 'phi', 'v_r', 'v_phi',
       'source_id'],
      dtype='object')

In [5]:
# Generate covariance matrix in ICRS
C_icrs = cov.generate_covmat(icrs_data)

In [30]:
# Export data to GPU

trans_needed_columns = ['source_id', 'ra', 'dec', 'r_est', 'pmra', 'pmdec', 'radial_velocity',]
icrs_gpu = cp.asarray(icrs_data[trans_needed_columns], dtype=cp.float32)

galcen_gpu = cp.asarray(galcen_data, dtype=cp.float32)
galcen_gpu.dtype

C_icrs_gpu = cp.asarray(C_icrs, dtype=cp.float32)

# Coordinate Transformation

## Coordinate Transformation Prototype

In [22]:
def get_H_matrix_gpu(Z_0, R_0):
    
    if(Z_0/R_0 is None):
        print("Something went wrong! No values for either Z_0 or R_0 were found!")
        return
   
    THETA_0 = np.arcsin(Z_0/R_0)

    # Defining constants to reduce process time
    costheta = np.cos(THETA_0)
    sintheta = np.sin(THETA_0)

    H = cp.array([[costheta, 0, sintheta],
                 [0, 1, 0],
                 [-sintheta, 0, costheta]], dtype=cp.float32)

    return H

def get_b_matrix_gpu(ra, dec):

    #B = np.array([[cosra*cosdec, -sinra, -cosra*sindec],
    #         [sinra*cosdec, cosra, -sinra*sindec],
    #         [sindec, np.zeros(n), cosdec]])
    
    
    # Add check if ra == dec
    
    n = len(ra)

    B = cp.zeros((n, 3, 3), dtype=cp.float32)
    
    # Defining constants to reduce process time
    cosra = np.cos(ra)
    cosdec = np.cos(dec)
    sinra = np.sin(ra)
    sindec = np.sin(dec)

    B[:, 0, 0] = cosra*cosdec
    B[:, 0, 1] = -sinra
    B[:, 0, 2] = -cosra*sindec
    
    B[:, 1, 0] = sinra*cosdec
    B[:, 1, 1] = cosra
    B[:, 1, 2] = -sinra*sindec
    
    B[:, 2, 0] = sindec
    B[:, 2, 2] = cosdec
    
    # Returns array of B matrices - one for each data point
    return B


def get_cylindrical_velocity_matrix_gpu(phi):
   
    n = len(phi)

    sin_phi = np.sin(phi).ravel()
    cos_phi = np.cos(phi).ravel()

    M = cp.array([[cos_phi, sin_phi, cp.zeros(n)],
         [-sin_phi, cos_phi, cp.zeros(n)],
         [cp.zeros(n), cp.zeros(n), cp.ones(n)]], dtype=cp.float32)

    M = M.T.reshape(n,3,3, order = 'A').swapaxes(1,2)

    # Returns array of matrices - one for each data point
    return M

In [40]:
from transformation_constants import A, k1, k2

A = cp.asarray(A, dtype=cp.float32)

def transform_coordinates_galactocentric_gpu(data_icrs, 
                                        z_0 = transformation_constants.Z_0, 
                                        r_0 = transformation_constants.R_0, 
                                        is_bayes = False):
    """This function uses input ICRS data and outputs data in cartesian (x,y,z) coordinates and in galactocentric frame of reference.

    Args:
        data_icrs (DataFrame): DataFrame of ICRS coordinates
        z_0 (float, optional): Sun's height over Galactic plane. Defaults to transformation_constants.Z_0.
        r_0 (float, optional): Sun's Galactocentric distance. Defaults to transformation_constants.R_0.
        is_bayes (bool, optional): Flag for using pre-computed (Bayesian) distance estimates. Defaults to False.

    Returns:
        ndarray: Array of Cartesian coordinates of shape (n,3,1).
    """

    #TODO: Add ASSERT checks on function input parameters.
    # ra dec can only be in a specific range

    # Number of data points
    n = len(data_icrs)

    # Going from DEG -> RAD
    ra = np.deg2rad(data_icrs[:,1])
    dec = np.deg2rad(data_icrs[:,2])

    if(is_bayes):
        c1 = data_icrs[:,3]

    else:
        # from kpc -> pc
        k1 = transformation_constants.k1

        # Declaring constants to reduce process time
        c1 = k1/data_icrs[:,3]

    cosdec = np.cos(dec)

    # Initial cartesian coordinate vector in ICRS
    coordxyz_ICRS = cp.asarray([[(c1)*np.cos(ra)*cosdec],
                      [(c1)*np.sin(ra)*cosdec],
                       [(c1)*np.sin(dec)]])

    coordxyz_ICRS = coordxyz_ICRS.T.reshape(n,3,1, order = 'A')

    # Using M1, M2, M3 for transparency in case of bugs
    M1 = cp.matmul(A,coordxyz_ICRS)
    M2 = M1 - cp.asarray([[r_0],
                        [0],
                        [0]], dtype=cp.float32)

    M3 = cp.matmul(get_H_matrix_gpu(z_0, r_0),M2)
   

    # Return is a np.array of shape (n,3,1)
    return M3

'''
This function uses input ICRS data and outputs data in cartesian (v_x,v_y,v_z) velocity vector components and in galactocentric frame of reference.
'''
def transform_velocities_galactocentric(data_icrs, 
                                z_0 = transformation_constants.Z_0, 
                                r_0 = transformation_constants.R_0, 
                                v_sun = transformation_constants.V_SUN, 
                                is_bayes = False):
    """Function for transforming proper motions with radial velocities to Cartesian velocity vector components in galactocentric frame.

    Args:
        data_icrs (DataFrame): DataFrame in ICRS
        z_0 (float, optional): Sun's position over Galactic plane. Defaults to transformation_constants.Z_0.
        r_0 (float, optional): Sun's Galactocentric distance. Defaults to transformation_constants.R_0.
        v_sun (tuple, optional): Sun's velocity vector. Defaults to transformation_constants.V_SUN.
        is_bayes (bool, optional): Flag for using pre-computed (Bayesian) distance estimates. Defaults to False.

    Returns:
        ndarray: Cartesian velocity components of shape (n,3,1)
    """
    # Number of data points
    n = len(data_icrs)

    # Going from DEG -> RAD
    ra = np.deg2rad(data_icrs[:,1])
    dec = np.deg2rad(data_icrs[:,2])

    # from 1/yr -> km/s
    k2 = transformation_constants.k2

    if(is_bayes):
        # Assign r estimates to c2
        c2 = k2*(data_icrs[:,3]/1000)

    else:
        # Declaring constants to reduce process time
        c2 = k2/data_icrs[:,3]

    # Initial velocity vector in ICRS in units km/s
    v_ICRS = cp.asarray([[data_icrs[:,6]],
                      [(c2)*data_icrs[:,4]],
                      [(c2)*data_icrs[:,5]]])

    v_ICRS = v_ICRS.T.reshape(n,3,1, order = 'A')

    B = get_b_matrix_gpu(ra, dec)
    B = B.reshape(n,3,3, order = 'A')

    # Using M1, M2, M3, .. for transparency in case of bugs
    M2 = cp.matmul(A, cp.matmul(B,v_ICRS))
    M3 = cp.matmul(get_H_matrix_gpu(z_0, r_0), M2)

    # Return is a np.array of shape (n,3,1)
    M4 = M3 + cp.asarray(v_sun, dtype=cp.float32)
    return M4

def transform_velocities_cylindrical(velocities_xyz, phi):
    """Transforms Cartesian velocities to cylindrical

    Args:
        velocities_xyz (np.array): Cartesian velocity array
        phi (np.array): Array of phi coordinates

    Returns:
        np.array: Array of cylindrical velocity components.
    """
    v_cylindrical = cp.matmul(get_cylindrical_velocity_matrix_gpu(phi), velocities_xyz)

    return v_cylindrical


def get_transformed_data(data_icrs,
                        include_cylindrical = False,
                        z_0 = transformation_constants.Z_0,
                        r_0 = transformation_constants.R_0,
                        v_sun = transformation_constants.V_SUN,
                        debug = False,
                        is_source_included = False,
                        is_bayes = False):
    
    # Coordinate vector in galactocentric frame in xyz
    coords =  transform_coordinates_galactocentric_gpu(data_icrs, z_0, r_0, is_bayes)
    # # Velocity vector in galactocentric frame in xyz
    velocities = transform_velocities_galactocentric(data_icrs, z_0, r_0, v_sun, is_bayes)
    
    if(include_cylindrical):

        # Using arctan2 which is defined in range [-pi ; pi]
        phi = np.arctan2(coords[:,1],coords[:,0])
        vel_cyl = transform_velocities_cylindrical(velocities, phi)
        coords_cyl = (np.sqrt(coords[:,0]**2 + coords[:,1]**2), phi)
        
    galcen_out = cp.concatenate((cp.squeeze(coords, axis=2), cp.squeeze(velocities, axis=2)), axis=1)
    coords_cyl = cp.squeeze(cp.asarray(coords_cyl).T, axis=0)
    vel_cyl = cp.squeeze(vel_cyl, axis=2)[:,0:2]
    galcen_out = cp.concatenate((galcen_out, coords_cyl, vel_cyl), axis=1)
    
    return galcen_out

In [54]:
galcen_data_gpu = get_transformed_data(icrs_gpu,
                                       include_cylindrical = True,
                                       z_0 = z_0,
                                       r_0 = r_0,
                                       v_sun = v_sun,
                                       debug = True,
                                       is_bayes = True,
                                       is_source_included = True)
galcen_data_gpu.dtype

dtype('float32')

In [7]:
galcen_data = data_analysis.get_transformed_data(icrs_data,
                                       include_cylindrical = True,
                                       z_0 = z_0,
                                       r_0 = r_0,
                                       v_sun = v_sun,
                                       debug = True,
                                       is_bayes = True,
                                       is_source_included = True)

Starting galactocentric transformation loop over all data points.. 
Time elapsed for data coordinate transformation: 0.6289024209991112 sec


## Testing New Transformation

In [26]:
trans_needed_columns = ['source_id', 'ra', 'dec', 'r_est', 'pmra', 'pmdec', 'radial_velocity']
icrs_cpu = icrs_data[trans_needed_columns].to_numpy()
icrs_gpu = cp.asarray(icrs_data[trans_needed_columns], dtype=cp.float32)

# Old function
galcen_data = data_analysis.get_transformed_data(icrs_data,
                                       include_cylindrical = True,
                                       z_0 = z_0,
                                       r_0 = r_0,
                                       v_sun = v_sun,
                                       debug = True,
                                       is_bayes = True,
                                       is_source_included = True)

# New function
NUMPY_LIB = np
galcen_data_new_cpu = transformation_functions.get_transformed_data(icrs_cpu,
                                       include_cylindrical = True,
                                       z_0 = z_0,
                                       r_0 = r_0,
                                       v_sun = v_sun,
                                       debug = True,
                                       is_bayes = True,
                                       is_source_included = True, 
                                       NUMPY_LIB = NUMPY_LIB)



NUMPY_LIB = cp
dtype = cp.float32
galcen_data_new_gpu = transformation_functions.get_transformed_data(icrs_gpu,
                                       include_cylindrical = True,
                                       z_0 = z_0,
                                       r_0 = r_0,
                                       v_sun = v_sun,
                                       debug = True,
                                       is_bayes = True,
                                       is_source_included = True, 
                                       NUMPY_LIB = NUMPY_LIB,
                                       dtype = dtype)

Starting galactocentric transformation loop over all data points.. 
Time elapsed for data coordinate transformation: 0.6237074750001739 sec


In [13]:
print(galcen_data.iloc[0].to_numpy())
print(galcen_data_new_cpu[0])
print(galcen_data_new_gpu[0])

print(galcen_data.shape)
print(galcen_data_new_cpu.shape)
print(galcen_data_new_gpu.shape)

print(galcen_data.dtypes)
print(galcen_data_new_cpu.dtype)
print(galcen_data_new_gpu.dtype)

[-8.39269568e+03  8.08717082e+00 -1.03851431e+02  8.24023689e+01
  1.74493940e+02  3.83473567e+01  8.39269958e+03  3.14062906e+00
 -8.22341890e+01 -1.74573262e+02  1.32667246e+14]
[-8.39269568e+03  8.08717082e+00 -1.03851431e+02  8.24023689e+01
  1.74493940e+02  3.83473567e+01  8.39269958e+03  3.14062906e+00
 -8.22341890e+01 -1.74573262e+02]
[-8.39269629e+03  8.08716774e+00 -1.03851425e+02  8.24023743e+01
  1.74493942e+02  3.83473511e+01  8.39269922e+03  3.14062905e+00
 -8.22341919e+01 -1.74573273e+02]
(1694972, 11)
(1694972, 10)
(1694972, 10)
x            float64
y            float64
z            float64
v_x          float64
v_y          float64
v_z          float64
r            float64
phi          float64
v_r          float64
v_phi        float64
source_id      int64
dtype: object
float64
float32


In [13]:
icrs_data.columns

Index(['source_id', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax',
       'parallax_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error',
       'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr',
       'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr',
       'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr',
       'radial_velocity', 'radial_velocity_error', 'r_est'],
      dtype='object')

## Profiling Transformation

In [40]:
def tranf_func_gpu():

    galcen_data_gpu = transformation_functions.get_transformed_data(icrs_gpu,
                                       include_cylindrical = True,
                                       z_0 = z_0,
                                       r_0 = r_0,
                                       v_sun = v_sun,
                                       debug = True,
                                       is_bayes = True,
                                       is_source_included = True, 
                                       NUMPY_LIB = NUMPY_LIB,
                                       dtype = dtype)


In [41]:
def tranf_func():
    galcen_data = data_analysis.get_transformed_data(icrs_data,
                                       include_cylindrical = True,
                                       z_0 = z_0,
                                       r_0 = r_0,
                                       v_sun = v_sun,
                                       debug = True,
                                       is_bayes = True,
                                       is_source_included = True)

In [51]:
cProfile.run('tranf_func_gpu()')

here
         1000 function calls in 0.007 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.007    0.007 797637153.py:1(tranf_func_gpu)
       16    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(can_cast)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(min_scalar_type)
        1    0.000    0.000    0.007    0.007 <string>:1(<module>)
        6    0.000    0.000    0.000    0.000 _gufuncs.py:194(_determine_from_args)
       18    0.000    0.000    0.000    0.000 _gufuncs.py:196(<genexpr>)
        6    0.000    0.000    0.000    0.000 _gufuncs.py:253(determine_dtype)
        6    0.000    0.000    0.002    0.000 _gufuncs.py:390(_apply_func_to_inputs)
       12    0.000    0.000    0.000    0.000 _gufuncs.py:416(_transpose_element)
       36    0.000    0.000    0.000    0.000 _gufuncs.py:417(<genexpr>)
       20    0.000    0.000    0.000 

In [47]:
cProfile.run('tranf_func()')

Starting galactocentric transformation loop over all data points.. 
here
Time elapsed for data coordinate transformation: 0.6294019090018992 sec
         6837 function calls (6729 primitive calls) in 0.630 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.629    0.629 4055506708.py:1(tranf_func)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
        1    0.000    0.000    0.002    0.002 <__array_function__ internals>:177(array_equal)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
        5    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(concatenate)
       16    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(copyto)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(insert)
        1    0.000    0.000    0.000    0.000 <__array_function

# Covariance Transformation

## Covariance Transformation Prototype

In [7]:
from transformation_constants import A, k1, k2

A = cp.asarray(A, dtype=cp.float32)

def get_jacobian_bayes_gpu(df, coordinate_system, Z_0, R_0):

    n = len(df)
    
    if(coordinate_system == "Cartesian"):

        if(Z_0/R_0 is None):
                print("Something went wrong! No values for either Z_0 or R_0 were found!")
                return
    
        THETA_0 = cp.arcsin(Z_0/R_0, dtype = cp.float32)

        # TODO: Implement exception handling!

        # DF -> ["ra", "dec","r_est","pmra","pmdec","radial_velocity"]
        r_est = df[:,2]

        c1 = r_est
        c2 = cp.float32(1)
        c3 = k2*(r_est/1000)
        c4 = -k2/1000

        ra = df[:,0]
        dec = df[:,1]
        mu_ra = df[:,3]
        mu_dec = df[:,4]
        v_r = df[:,5]

        # deg -> radians
        ra = cp.deg2rad(ra)
        dec = cp.deg2rad(dec)

        # Declaring variables to reduce number of computations 
        sin_ra = cp.sin(ra)
        cos_ra = cp.cos(ra)
        sin_dec = cp.sin(dec)
        cos_dec = cp.cos(dec)
        sin_theta = cp.sin(THETA_0)
        cos_theta = cp.cos(THETA_0)
        
        A_1 = A[0,0]*cos_theta + A[2,0]*sin_theta
        A_2 = A[0,1]*cos_theta + A[2,1]*sin_theta
        A_3 = A[0,2]*cos_theta + A[2,2]*sin_theta
        A_4 = -A[1,0]*sin_ra + A[1,1]*cos_ra
        A_5 = A[1,0]*cos_ra + A[1,1]*sin_ra
        A_6 = A[2,2]*cos_theta - A[0,2]*sin_theta
        A_7 = A[2,0]*cos_theta - A[0,0]*sin_theta
        A_8 = A[2,1]*cos_theta - A[0,1]*sin_theta
        
        print(THETA_0.dtype)
        cosra_cosdec = cos_ra*cos_dec
        cosra_sindec = cos_ra*sin_dec
        sinra_cosdec = sin_ra*cos_dec
        sinra_sindec = sin_ra*sin_dec

        expr_1 = cos_dec*v_r - sin_dec*c3*mu_dec
        #expr_2 = -cos_dec*v_r + sin_dec*c3*mu_dec
        expr_3 = -sin_dec*v_r - cos_dec*c3*mu_dec
        expr_4 = sin_ra*c4*mu_ra + cosra_sindec*c4*mu_dec
        #expr_5 = cos_ra*c3*mu_ra
        #expr_6 = sin_ra*c3*mu_ra
        expr_7 = -cos_ra*c4*mu_ra + sinra_sindec*c4*mu_dec
        expr_8 = -sin_ra*v_r + c3*(cos_dec**(-1))*(-cos_ra*mu_ra + sinra_sindec*mu_dec)
        expr_9 = cos_ra*v_r - c3*(cos_dec**(-1))*(sin_ra*mu_ra + cosra_sindec*mu_dec)

        

        J11 = c1*(-sin_ra*(A_1) + cos_ra*(A_2))
        J12 = c1*(cos_dec*(A_3) - sin_dec*(cos_ra*(A_1) + sin_ra*(A_2)))
        J13 = c2*(cosra_cosdec*(A_1) + sinra_cosdec*(A_2) + sin_dec*(A_3))
        J14 = cp.zeros(n, dtype=cp.float32)
        #J15 = np.zeros(n)
        #J16 = np.zeros(n)
        
        #row_1 = np.array([J11, J12, J13, J14, J15, J16])
        
        J21 = c1*(A_4) 
        J22 = c1*(-sin_dec*A_5 + A[1,2]*cos_dec) 
        J23 = c2*(cos_dec*A_5 + A[1,2]*sin_dec)
        # J24 = np.zeros(n)
        # J25 = np.zeros(n)
        # J26 = np.zeros(n)

        #J31 = c1*(-sin_theta*(A[0,1]*cos_ra - A[0,0]*sin_ra) + cos_theta*(A[2,1]*cos_ra - A[2,0]*sin_ra))
        J31 = c1*(-sin_ra*(A_7) + cos_ra*(A_8))
        J32 = -c1*(sin_dec*(cos_ra*(A[0,0]*sin_theta - A[2,0]*cos_theta) + sin_ra*(A[0,1]*sin_theta - A[2,1]*cos_theta)) + cos_dec*(A_6))
        J33 = c2*(cosra_cosdec*(A_7) + sinra_cosdec*(A_8) + sin_dec*(A_6))     
        # J34 = np.zeros(n)
        # J35 = np.zeros(n)
        # J36 = np.zeros(n)

        #J41 = (sin_ra*(expr_2) - expr_5)*(A_1) + (cos_ra*(expr_1) - expr_6)*(A_2)

        J41 = (expr_8)*(A_1) + (expr_9)*(A_2)
        J42 = cos_ra*(expr_3)*(A_1) + sin_ra*(expr_3)*(A_2) + (expr_1)*(A_3)
        J43 = (expr_4)*(A_1) + (expr_7)*(A_2) + (-cos_dec*c4*mu_dec)*(A_3)
        J44 = -sin_ra*c3*(A_1) + cos_ra*c3*(A_2)
        J45 = (-cosra_sindec*c3)*(A_1) + (-sinra_sindec*c3)*(A_2) + (cos_dec*c3)*(A_3)
        J46 = cosra_cosdec*(A_1) + sinra_cosdec*(A_2) + sin_dec*(A_3)

        #J51 = A[1,0]*(-sinra_cosdec*v_r - expr_5 + sinra_sindec*c3*mu_dec) + A[1,1]*(cosra_cosdec*v_r - expr_6- cosra_sindec*c3*mu_dec)
        J51 = A[1,0]*(expr_8) + A[1,1]*(expr_9)
        J52 = A[1,0]*cos_ra*(expr_3) + A[1,1]*sin_ra*(expr_3) + A[1,2]*(expr_1)
        J53 = A[1,0]*(expr_4) + A[1,1]*(expr_7) + A[1,2]*(-cos_dec*c4*mu_dec)
        J54 = c3*(A_4)
        J55 = c3*(-A[1,0]*cosra_sindec- A[1,1]*sinra_sindec + A[1,2]*cos_dec)
        J56 = (A[1,0]*cos_ra + A[1,1]*sin_ra)*cos_dec + A[1,2]*sin_dec

        #J61 = (sin_ra*(expr_2) - expr_5)*(A_7) + (cos_ra*(expr_1) - expr_6)*(A_8)
        J61 = (expr_8)*(A_7) + (expr_9)*(A_8)
        J62 = cos_ra*(expr_3)*(A_7) + sin_ra*(expr_3)*(A_8) + (expr_1)*(A_6)
        J63 = c4*((sin_ra*mu_ra + cosra_sindec*mu_dec)*(A_7) + (-cos_ra*mu_ra + sinra_sindec*mu_dec)*(A_8) - cos_dec*mu_dec*(A_6))
        J64 = c3*(-sin_ra*(A_7) + cos_ra*(A_8))
        J65 = c3*(-cosra_sindec*(A_7) - sinra_sindec*(A_8) + cos_dec*(A_6))
        J66 = cosra_cosdec*(A_7) + sinra_cosdec*(A_8) + sin_dec*(A_6)
        
        

        J1 = cp.stack((J11, J12, J13, J14, J14, J14))
        J2 = cp.stack((J21, J22, J23, J14, J14, J14))
        J3 = cp.stack((J31, J32, J33, J14, J14, J14))
        J4 = cp.stack((J41, J42, J43, J44, J45, J46))
        J5 = cp.stack((J51, J52, J53, J54, J55, J56))
        J6 = cp.stack((J61, J62, J63, J64, J65, J66))
        
        J = cp.stack((J1, J2, J3, J4, J5, J6))
       
    return J


def get_jacobian_gpu(df, coordinate_system, Z_0, R_0):

    n = len(df)
    
    if(coordinate_system == "Cylindrical"):

        # TODO: Implement exception handling!

        # DF -> ["x", "y","r","phi","v_r","v_phi"]

        x = df[:,0]
        y = df[:,1]
        r = df[:,2]
        phi = df[:,3]
        v_r = df[:,4]
        v_phi = df[:,5]


    
        c1 = x/(r**2)
        c2 = y/(r**2)
        
        # Declaring variables to reduce number of computations 
        sin_phi = cp.sin(phi)
        cos_phi = cp.cos(phi)

        J11 = x/r
        J12 = y/r
        J13 = cp.zeros(n, dtype=cp.float32)
        #J14 = np.zeros(n)
        #J15 = np.zeros(n)
        #J16 = np.zeros(n)
    
        J21 = -c2 
        J22 = c1
        #J23 = np.zeros(n)
        #J24 = np.zeros(n)
        #J25 = np.zeros(n)
        #J26 = np.zeros(n)

        #J31 = np.zeros(n)
        #J32 = np.zeros(n)
        J33 = cp.ones(n, dtype=cp.float32)
        #J34 = np.zeros(n)
        #J35 = np.zeros(n)
        #J36 = np.zeros(n)

        J41 = -v_phi*c2
        J42 = v_phi*c1
        #J43 = np.zeros(n)
        J44 = cos_phi
        J45 = sin_phi
        #J46 = np.zeros(n)

        J51 = v_r*c2
        J52 = -v_r*c1
        #J53 = np.zeros(n) 
        J54 = -sin_phi
        J55 = cos_phi
        #J56 = np.zeros(n)

        #J61 = np.zeros(n)
        #J62 = np.zeros(n)
        #J63 = np.zeros(n)
        #J64 = np.zeros(n)
        #J65 = np.zeros(n)
        #J66 = np.ones(n)
               
        J1 = cp.stack((J11, J12, J13, J13, J13, J13))
        J2 = cp.stack((J21, J22, J13, J13, J13, J13))
        J3 = cp.stack((J13, J13, J33, J13, J13, J13))
        J4 = cp.stack((J41, J42, J13, J44, J45, J13))
        J5 = cp.stack((J51, J52, J13, J54, J55, J13))
        J6 = cp.stack((J13, J13, J13, J13, J13, J33))
        
        J = cp.stack((J1, J2, J3, J4, J5, J6))
        
    return J


def transform_cov_matrix_gpu(C, 
                        df, 
                        coordinate_system, 
                        z_0 = transformation_constants.Z_0, 
                        r_0 = transformation_constants.R_0, 
                        is_bayes = False):
    """Transforms an array of covariance matrices to specified coordinate system.

    Args:
        C (Array): Arrays of covariance matrices
        df (DataFrame): DataFrame of Gaia data used for generating Jacobian matrices for each observation.
        coordinate_system (str): Specified coordinate system, either "Cartesian" or "Cylindrical".
        z_0 (float, optional): Sun's height over Galactic plane. Defaults to transformation_constants.Z_0.
        r_0 (float, optional): Sun's distance from Galactic centre. Defaults to transformation_constants.R_0.
        is_bayes (bool, optional): Set True if using distance estimates instead of parallaxes. Defaults to False.

    Returns:
        Array: Returns array of transformed covariance matrices.
    """

    if(is_bayes == True):
        # Grabs the correct Jacobian for every point in data set. Of shape (n, 6, 6).
        J = get_jacobian_bayes_gpu(df, 
                                coordinate_system, 
                                Z_0 = z_0, 
                                R_0 = r_0)
        print(J.dtype)

    else:
        # Grabs the correct Jacobian for every point in data set. Of shape (n, 6, 6).
        J = get_jacobian_gpu(df, 
                            coordinate_system, 
                            Z_0 = z_0, 
                            R_0 = r_0)


    J = J.T.reshape(len(df), 6, 6, order = 'A').swapaxes(1,2)
    J_trunc= J.reshape(len(df),6,6, order = 'A').swapaxes(1,2)

    C_transformed_gpu = cp.matmul(cp.matmul(J, C), J_trunc)

    return C_transformed_gpu

In [22]:
def transform_cov_matrix(C, 
                        df, 
                        coordinate_system, 
                        z_0 = transformation_constants.Z_0, 
                        r_0 = transformation_constants.R_0, 
                        is_bayes = False):
    """Transforms an array of covariance matrices to specified coordinate system.

    Args:
        C (Array): Arrays of covariance matrices
        df (DataFrame): DataFrame of Gaia data used for generating Jacobian matrices for each observation.
        coordinate_system (str): Specified coordinate system, either "Cartesian" or "Cylindrical".
        z_0 (float, optional): Sun's height over Galactic plane. Defaults to transformation_constants.Z_0.
        r_0 (float, optional): Sun's distance from Galactic centre. Defaults to transformation_constants.R_0.
        is_bayes (bool, optional): Set True if using distance estimates instead of parallaxes. Defaults to False.

    Returns:
        Array: Returns array of transformed covariance matrices.
    """

    if(is_bayes == True):
        # Grabs the correct Jacobian for every point in data set. Of shape (n, 6, 6).
        J = transformation_constants.get_jacobian_bayes(df, 
                                                        coordinate_system, 
                                                        Z_0 = z_0, 
                                                        R_0 = r_0)
    else:
        # Grabs the correct Jacobian for every point in data set. Of shape (n, 6, 6).
        J = transformation_constants.get_jacobian(df, 
                                                coordinate_system, 
                                                Z_0 = z_0, 
                                                R_0 = r_0)

    J = J.T.reshape(len(df), 6, 6, order = 'A').swapaxes(1,2)

    J_trunc= J.reshape(len(df),6,6, order = 'A').swapaxes(1,2)

    C_transformed = J @ C @ J_trunc

    return C_transformed

## Testing Covariance Transformation
### Galactocentric Covariance Transformation

In [14]:
# Gets columns after 'source_id'
# ["ra", "dec","r_est","pmra","pmdec","radial_velocity"]
data_array = icrs_gpu[:,1::]
data_array.dtype

galactocentric_cov = transform_cov_matrix_gpu(C_icrs_gpu, 
                                        data_array, 
                                        "Cartesian", 
                                        z_0=z_0, 
                                        r_0=r_0, 
                                        is_bayes=True)
galactocentric_cov.dtype

NameError: name 'transform_cov_matrix_gpu' is not defined

### Cylindrical Covariance Transformation

In [9]:
# Gets correct columns for next transformation
#["x", "y","r","phi","v_r","v_phi"]
data_array_galcen = galcen_gpu[:,[0,1,6,7,8,9]]

cyl_cov_gpu = transform_cov_matrix_gpu(galactocentric_cov, 
                                        data_array_galcen, 
                                        "Cylindrical", 
                                        z_0=z_0, 
                                        r_0=r_0)
cyl_cov_gpu.dtype

NameError: name 'galcen_gpu' is not defined

In [10]:
# Generate covariance matrix in ICRS
C_icrs = cov.generate_covmat(icrs_data)
C_icrs_gpu = cp.asarray(C_icrs, dtype=cp.float32)

trans_needed_columns = ['source_id', 'ra', 'dec', 'r_est', 'pmra', 'pmdec', 'radial_velocity']

icrs_cpu = icrs_data[trans_needed_columns].to_numpy()
icrs_gpu = cp.asarray(icrs_data[trans_needed_columns], dtype=cp.float32)

data_array_cpu = icrs_cpu[:,1::]
data_array_gpu = icrs_gpu[:,1::]

data_array_galcen_cpu = galcen_data_new_cpu[:,[0,1,6,7,8,9]]
data_array_galcen_gpu = galcen_data_new_gpu[:,[0,1,6,7,8,9]]

# Old function
galactocentric_cov = cov.transform_cov_galactocentric(df = icrs_data, 
                                                        C = C_icrs,
                                                        is_bayes = True,
                                                        Z_0 = z_0,
                                                        R_0 = r_0)

cyl_cov = cov.transform_cov_cylindirical(df_crt = galcen_data, 
                                        C = galactocentric_cov,
                                        Z_0 = z_0,
                                        R_0 = r_0)

# CPU TESTING SECTION -------------------------------------------------------->
# New function
NUMPY_LIB = np
galactocentric_cov_cpu = cov.transform_cov_matrix(C = C_icrs, 
                                                df = data_array_cpu,
                                                coordinate_system = 'Cartesian',
                                                z_0 = z_0,
                                                r_0 = r_0,
                                                is_bayes = True,
                                                NUMPY_LIB = NUMPY_LIB)

cyl_cov_cpu = cov.transform_cov_matrix(C = galactocentric_cov_cpu, 
                                    df = data_array_galcen_cpu,
                                    coordinate_system = 'Cylindrical',
                                    z_0 = z_0,
                                    r_0 = r_0,
                                    is_bayes = False,
                                    NUMPY_LIB = NUMPY_LIB)


# GPU TESTING SECTION -------------------------------------------------------->
NUMPY_LIB = cp
dtype = cp.float32
galactocentric_cov_gpu = cov.transform_cov_matrix(C = C_icrs_gpu, 
                                                df = data_array_gpu,
                                                coordinate_system = 'Cartesian',
                                                z_0 = z_0,
                                                r_0 = r_0,
                                                is_bayes = True,
                                                NUMPY_LIB = NUMPY_LIB,
                                                dtype = dtype)

cyl_cov_gpu = cov.transform_cov_matrix(C = galactocentric_cov_gpu, 
                                    df = data_array_galcen_gpu,
                                    coordinate_system = 'Cylindrical',
                                    z_0 = z_0,
                                    r_0 = r_0,
                                    is_bayes = False,
                                    NUMPY_LIB = NUMPY_LIB,
                                    dtype = dtype)

In [7]:
print(cyl_cov.iloc[0].sig_vphi)
print(cyl_cov_cpu[0, 4, 4])
print(cyl_cov_gpu[0, 4, 4])

print(cyl_cov.shape)
print(cyl_cov_cpu.shape)
print(cyl_cov_gpu.shape)

print(cyl_cov.dtypes)
print(cyl_cov_cpu.dtype)
print(cyl_cov_gpu.dtype)

0.007613203980363394
0.007613203980363394
0.0076132035
(100, 3)
(100, 6, 6)
(100, 6, 6)
source_id      int64
sig_vphi     float64
sig_vr       float64
dtype: object
float64
float32


In [5]:
final_columns = ['x', 'y', 'z', 'v_x', 'v_y', 'v_z', 'r', 'phi', 'v_r', 'v_phi',
       'sig_vphi', 'sig_vr', 'source_id']

In [11]:
sig_vphi = cp.array([cyl_cov_gpu[:,4,4]])
sig_vr = cp.array([cyl_cov_gpu[:,3,3]])
source_id = cp.array([icrs_gpu[:,0]])
df_test = cp.concatenate(([galcen_data_new_gpu, sig_vphi.T, sig_vr.T, source_id.T]), axis=1)


# OUTSIDE FUNC
pandas_frame = pd.DataFrame(df_test.get(), columns=final_columns)
pandas_frame.shape

(1000000, 13)

## Profiling Covariance Transformation

In [16]:
def cov_func_gpu():
    galactocentric_cov_gpu = cov.transform_cov_matrix(C = C_icrs_gpu, 
                                                df = data_array_gpu,
                                                coordinate_system = 'Cartesian',
                                                z_0 = z_0,
                                                r_0 = r_0,
                                                is_bayes = True,
                                                NUMPY_LIB = NUMPY_LIB,
                                                dtype = dtype)

    cyl_cov_gpu = cov.transform_cov_matrix(C = galactocentric_cov_gpu, 
                                    df = data_array_galcen_gpu,
                                    coordinate_system = 'Cylindrical',
                                    z_0 = z_0,
                                    r_0 = r_0,
                                    is_bayes = False,
                                    NUMPY_LIB = NUMPY_LIB,
                                    dtype = dtype)

    

In [17]:
def cov_func():

    # Old function
    galactocentric_cov = cov.transform_cov_galactocentric(df = icrs_data, 
                                                            C = C_icrs,
                                                            is_bayes = True,
                                                            Z_0 = z_0,
                                                            R_0 = r_0)

    cyl_cov = cov.transform_cov_cylindirical(df_crt = galcen_data, 
                                            C = galactocentric_cov,
                                            Z_0 = z_0,
                                            R_0 = r_0)


In [29]:
cProfile.run('cov_func_gpu()')

         1434 function calls in 0.011 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.011    0.011 1581584888.py:1(cov_func_gpu)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(can_cast)
       16    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(min_scalar_type)
        1    0.000    0.000    0.011    0.011 <string>:1(<module>)
        4    0.000    0.000    0.000    0.000 _gufuncs.py:194(_determine_from_args)
       12    0.000    0.000    0.000    0.000 _gufuncs.py:196(<genexpr>)
        4    0.000    0.000    0.000    0.000 _gufuncs.py:253(determine_dtype)
        4    0.000    0.000    0.001    0.000 _gufuncs.py:390(_apply_func_to_inputs)
        8    0.000    0.000    0.000    0.000 _gufuncs.py:416(_transpose_element)
       24    0.000    0.000    0.000    0.000 _gufuncs.py:417(<genexpr>)
       16    0.000    0.000    0.000    0.0

In [30]:
cProfile.run('cov_func()')

         2398 function calls (2365 primitive calls) in 1.937 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    1.936    1.936 2532643863.py:1(cov_func)
       14    0.000    0.000    0.209    0.015 <__array_function__ internals>:177(concatenate)
       10    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(copyto)
       14    0.000    0.000    0.209    0.015 <__array_function__ internals>:177(stack)
        6    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1033(_handle_fromlist)
        1    0.001    0.001    1.937    1.937 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 _asarray.py:111(<setcomp>)
        2    0.000    0.000    0.000    0.000 _asarray.py:22(require)
        4    0.000    0.000    0.000    0.000 _dtype.py:24(_kind_name)
        4    0.000    0.000    0.000    0.000 _dtype.py:314(_name_includes_bit_suffix)
        4    0.000

In [78]:
from cupyx.profiler import benchmark
def my_func(C_icrs, data_array, z_0, r_0):

    galactocentric_cov = transform_cov_matrix_gpu(C_icrs, 
                                        data_array, 
                                        "Cartesian", 
                                        z_0=z_0, 
                                        r_0=r_0, 
                                        is_bayes=True)

    return galactocentric_cov

print(benchmark(my_func, (C_icrs,data_array, z_0, r_0 ,), n_repeat=1))  

<class 'cupy.ndarray'>


TypeError: Argument 'array' has incorrect type (expected cupy._core.core._ndarray_base, got numpy.ndarray)

# Testing BinCollection

In [12]:
# # Generate bins
bin_collection = data_analysis.get_collapsed_bins(data = pandas_frame,
                                                      theta = (0, 1),
                                                      BL_r_min = 5000,
                                                      BL_r_max = 15000,
                                                      BL_z_min = -200,
                                                      BL_z_max = 200,
                                                      N_bins = (5, 1),
                                                      r_drift = False,
                                                      debug = False)

In [6]:
galactocentric_cov = cov.generate_galactocentric_covmat(icrs_data, 
                                                            is_bayes = True,
                                                            Z_0 = z_0,
                                                            R_0 = r_0)

cyl_cov = cov.transform_cov_cylindirical(galcen_data, 
                                             C = galactocentric_cov,
                                             Z_0 = z_0,
                                             R_0 = r_0)

galcen_data = galcen_data.merge(cyl_cov, on='source_id')

# # Generate bins
bin_collection = data_analysis.get_collapsed_bins(data = galcen_data,
                                                      theta = (0, 1),
                                                      BL_r_min = 5000,
                                                      BL_r_max = 15000,
                                                      BL_z_min = -200,
                                                      BL_z_max = 200,
                                                      N_bins = (5, 1),
                                                      r_drift = False,
                                                      debug = False)

In [16]:
galcen_data_trimmed = galcen_data[['r', 'z', 'v_r', 'v_phi',
       'source_id', 'sig_vphi', 'sig_vr']]

In [21]:

bin_collection = data_analysis.get_collapsed_bins(data = galcen_data,
                                                    theta = (0, 1),
                                                    BL_r_min = 5000,
                                                    BL_r_max = 15000,
                                                    BL_z_min = -200,
                                                    BL_z_max = 200,
                                                    N_bins = (5, 1),
                                                    r_drift = False,
                                                    debug = False)

In [7]:
galcen_data.shape

(37921, 11)

# Adimensional binning stuff

In [ ]:
r_0 = 8277

galcen_data = data_analysis.get_transformed_data(icrs_data,
                                       include_cylindrical = True,
                                       z_0 = z_0,
                                       r_0 = r_0,
                                       v_sun = v_sun,
                                       debug = True,
                                       is_bayes = True,
                                       is_source_included = True)

galcen_data = galcen_data[(galcen_data.r < 15000) & (galcen_data.r > 5000)]
galcen_data = galcen_data[(galcen_data.z < 200) & (galcen_data.z > -200)]
galcen_data.reset_index(inplace=True, drop=True)

In [ ]:
bin_collection_orig = data_analysis.get_collapsed_bins(data = galcen_data,
                                                    theta = (0, 1),
                                                    BL_r_min = 5000,
                                                    BL_r_max = 15000,
                                                    BL_z_min = -200,
                                                    BL_z_max = 200,
                                                    N_bins = (5, 1),
                                                    r_drift = False,
                                                    debug = False)





In [9]:
r_min = 5000/8277
print(r_min)
r_max = 15000/8277
print(r_max)

0.6040836051709556
1.812250815512867


In [ ]:
galcen_data['r_orig'] = galcen_data.r
galcen_data['r'] = galcen_data.r/8277 

In [ ]:
bin_collection_new = data_analysis.get_collapsed_bins(data = galcen_data,
                                                    theta = (0, 1),
                                                    BL_r_min = r_min,
                                                    BL_r_max = r_max,
                                                    BL_z_min = -200,
                                                    BL_z_max = 200,
                                                    N_bins = (5, 1),
                                                    r_drift = False,
                                                    debug = False)

# Bootstrapping

In [22]:
def bootstrap_weighted_error(bin_vphi, bin_sig_vphi):
    
    data_length = len(bin_vphi)
    idx_list = np.arange(data_length)
    bootstrapped_means = np.zeros(1000)

    for i in range(1000):
        rnd_idx = np.random.choice(idx_list, replace=True, size=data_length)
        test_sample = bin_vphi[rnd_idx]
        sig_vphi = bin_sig_vphi[rnd_idx]
        bootstrapped_means[i] = (test_sample/sig_vphi).sum()/(1/sig_vphi).sum()
    conf_int = np.percentile(bootstrapped_means, [16, 84])

    return (conf_int[1] - conf_int [0])/2

In [ ]:
cp.random.Generator.random

In [25]:
def bootstrap_weighted_error_gpu(bin_vphi, bin_sig_vphi):
    
    total_num_it = 1000
    batch_num = 10
    data_length = len(bin_vphi)
    idx_list = cp.arange(data_length)
    bootstrapped_means = cp.zeros(total_num_it)



    for i in range(100):
        rnd_idx = cp.random.choice(idx_list, replace=True, size=(batch_num, data_length))

        test_sample = bin_vphi[rnd_idx]
        sig_vphi = bin_sig_vphi[rnd_idx]

        start_idx = (i+1)*batch_num - batch_num
        end_idx = (i+1)*batch_num

        bootstrapped_means[start_idx:end_idx] = (test_sample/sig_vphi).sum(axis=1)/(1/sig_vphi).sum(axis=1)
    conf_int = cp.percentile(bootstrapped_means, [16, 84])

    return (conf_int[1] - conf_int [0])/2

In [20]:
# Testing suggestions

def bootstrap_weighted_error_gpu(bin_vphi, bin_sig_vphi):
    
    total_num_it = 1000
    batch_num = 10
    data_length = len(bin_vphi)
    idx_list = cp.arange(data_length)
    bootstrapped_means = cp.zeros(total_num_it)

    for i in range(100):
        rnd_idx = cp.random.choice(idx_list, replace=True, size=(batch_num, data_length))

        test_sample = bin_vphi[rnd_idx]
        sig_vphi = bin_sig_vphi[rnd_idx]

        start_idx = (i+1)*batch_num - batch_num
        end_idx = (i+1)*batch_num

        bootstrapped_means[start_idx:end_idx] = cp.divide(cp.sum(cp.divide(test_sample,sig_vphi),axis=1), cp.sum(cp.divide(1,sig_vphi),axis=1))

        #bootstrapped_means[start_idx:end_idx] = (test_sample/sig_vphi).sum(axis=1)/(1/sig_vphi).sum(axis=1)
    conf_int = cp.percentile(bootstrapped_means, [16, 84])

    return (conf_int[1] - conf_int [0])/2

In [19]:
for i, bin in enumerate(bin_collection.bins):
    a = bootstrap_weighted_error_gpu(cp.asarray(bin.data.v_phi), 
                                    cp.asarray(bin.data.sig_vphi))
    print(a)

    

0.08434672643578267
0.13636348952715593
0.09080084812264033
0.12972324922145617
0.3842372916217016


In [17]:
for i, bin in enumerate(bin_collection.bins):
    a = bootstrap_weighted_error_gpu(cp.asarray(bin.data.v_phi), 
                                    cp.asarray(bin.data.sig_vphi))
    print(a)

0.08497365171838567
0.14790581196704977
0.09694302198889204
0.1303011019374054
0.39777704305015504


In [35]:
# Fully vectorised
def bootstrap_weighted_error_gpu(bin_vphi, bin_sig_vphi):
    
    num_it = 1000
    data_length = len(bin_vphi)
    idx_list = cp.arange(data_length)
    bootstrapped_means = cp.zeros(num_it)

    rnd_idx = cp.random.choice(idx_list, replace=True, size=(num_it, data_length))
    
    test_sample = bin_vphi[rnd_idx]
    sig_vphi = bin_sig_vphi[rnd_idx]
    bootstrapped_means = (test_sample/sig_vphi).sum(axis=1)/(1/sig_vphi).sum(axis=1)
    conf_int = cp.percentile(bootstrapped_means, [16, 84])

    return (conf_int[1] - conf_int [0])/2

# Fully vectorised
def bootstrap_weighted_error_cpu(bin_vphi, bin_sig_vphi):
    
    num_it = 2
    data_length = len(bin_vphi)
    idx_list = np.arange(data_length)
    bootstrapped_means = np.zeros(num_it)
    rnd_idx = np.random.choice(idx_list, replace=True, size=(num_it,data_length))

    print(rnd_idx.shape)
    print(bin_vphi.shape)

    test_sample = bin_vphi[rnd_idx]
    sig_vphi = bin_sig_vphi[rnd_idx]
    bootstrapped_means = (test_sample/sig_vphi).sum(axis=1)/(1/sig_vphi).sum(axis=1)
    conf_int = np.percentile(bootstrapped_means, [16, 84])

    return (conf_int[1] - conf_int [0])/2

In [4]:
# for i, bin in enumerate(bin_collection.bins):
    # a = bootstrap_weighted_error_cpu(bin.data.v_phi, 
    #                                 bin.data.sig_vphi)
    # print(a)

In [42]:
for i, bin in enumerate(bin_collection.bins):
    a = bootstrap_weighted_error_gpu(cp.asarray(bin.data.v_phi), 
                                    cp.asarray(bin.data.sig_vphi))
    print(a)

OutOfMemoryError: Out of memory allocating 3,168,148,480 bytes (allocated so far: 5,504,759,808 bytes).

In [25]:
for i, bin in enumerate(bin_collection.bins):
    a = bootstrap_weighted_error(cp.asarray(bin.data.v_phi), 
                                    cp.asarray(bin.data.sig_vphi))
    print(a)

0.08398327623260116
0.1382662772271317
0.09862418241954174
0.1322435363425427
0.38986252724180304


In [26]:
def boot_func_gpu():
    for i, bin in enumerate(bin_collection.bins):
            a = bootstrap_weighted_error_gpu(cp.asarray(bin.data.v_phi), 
                                                cp.asarray(bin.data.sig_vphi))

In [37]:
def boot_func():
    for i, bin in enumerate(bin_collection.bins):
        a = bootstrap_weighted_error(bin.data.v_phi.to_numpy(), bin.data.sig_vphi.to_numpy())

In [28]:
cProfile.run('boot_func_gpu()')

         49684 function calls in 3.140 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.047    0.009    3.136    0.627 2088427909.py:1(bootstrap_weighted_error_gpu)
        1    0.000    0.000    3.140    3.140 989478649.py:1(boot_func_gpu)
     1375    0.001    0.000    0.003    0.000 <__array_function__ internals>:177(can_cast)
     2395    0.002    0.000    0.005    0.000 <__array_function__ internals>:177(min_scalar_type)
      500    0.000    0.000    0.007    0.000 <__array_function__ internals>:177(prod)
        1    0.000    0.000    3.140    3.140 <string>:1(<module>)
      500    0.013    0.000    3.035    0.006 _generator.py:1014(choice)
      500    0.009    0.000    3.012    0.006 _generator.py:1146(randint)
      500    0.001    0.000    0.001    0.000 _generator.py:1203(get_random_state)
      500    0.027    0.000    2.991    0.006 _generator.py:645(_interval)
      870    0.003    0.000    0.0

In [48]:
cProfile.run('boot_func()')

         125959 function calls (125929 primitive calls) in 34.177 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   34.177   34.177 1894174980.py:1(boot_func)
        5   14.655    2.931   34.177    6.835 493349493.py:1(bootstrap_weighted_error)
        5    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(any)
        5    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(concatenate)
        5    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(percentile)
    10000    0.009    0.000    0.093    0.000 <__array_function__ internals>:177(prod)
       15    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(take)
        5    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(unique)
        1    0.000    0.000   34.177   34.177 <string>:1(<module>)
    10000    0.006    0.000    0.744    0.000 _methods.py:46

In [27]:
from numba import jit, config
type(config.DISABLE_JIT)

int

In [24]:
from numba import jit, config
config.DISABLE_JIT = True

if(config.DISABLE_JIT == False):
    NUMPY_LIB = np

True

In [21]:
import numba
import math

@numba.extending.register_jitable
def sin_func(x, sin):
    return sin(x)

@numba.njit
def foo(x, sin_wrapper):
    return sin_func(x, sin_wrapper)

def sin_wrapper(x):
    return math.sin(x)

# Call the foo function with the sin_wrapper function as an argument
result = foo(0.5, sin_wrapper)
print(result)  # Outputs: 0.479425538604203

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
non-precise type pyobject
During: typing of argument at <ipython-input-21-3b7e2f516782> (10)

File "<ipython-input-21-3b7e2f516782>", line 10:
def foo(x, sin_wrapper):
    return sin_func(x, sin_wrapper)
    ^

This error may have been caused by the following argument(s):
- argument 1: Cannot determine Numba type of <class 'function'>
